# Import libraries and data

In [1]:
import pandas as pd
import numpy as np
import qgrid
import eli5

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, GroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit


import xgboost as xgb
import catboost as ctb


np.random.seed(0)

In [5]:
df_train = pd.read_hdf("../input/energy_train.h5")
df_test = pd.read_hdf("../input/energy_test.h5")
df_train["date"] = pd.to_datetime(df_train["date"], format = '%d/%m/%Y')
df_test["date"] = pd.to_datetime(df_test["date"], format = '%d/%m/%Y')

df_all = pd.concat([df_train, df_test])
df_all["date"] = pd.to_datetime(df_all["date"])

df_all.shape

(35252, 3)

# Feature engineering

In [7]:
def feature_engineering(df, d):
    df["day"] = df["date"].dt.day
    df["month"] = df["date"].dt.month
    df["hour"] = df["date"].dt.hour
    df["year"] = df["date"].dt.year
    df["week"] = df["date"].dt.week
    df["dayofweek"] = df["date"].dt.dayofweek
    df["dayofyear"] = df["date"].dt.dayofyear
    df["quarter"]=df["date"].dt.quarter    
    df["weekofyear"] = df["date"].dt.weekofyear   
    
    
    df.index = df["date"]
    df_train = df[ df.value.notnull() ]
    
    #get value from yesterday 
    #d = 1
    df_train_new = df_train.copy()
    feat_name = "dayofyear_{}d".format(d)
    value_feat_name = "value_{}dayofyear".format(d)

    df_train_new[feat_name] = df_train_new["dayofyear"] - d
    df_train_new.rename(columns={"value": value_feat_name}, inplace=True)
    
    df = pd.merge(df, df_train_new[ [value_feat_name, feat_name, "year", "hour"] ], left_on=["year", "dayofyear", "hour"], right_on=["year", feat_name, "hour"], how="left")
#     df = pd.merge(df, df_train_new2[ [value_feat_name2, feat_name2, "week", "hour"] ], left_on=["week", "dayofweek", "hour"], right_on=["week", feat_name2, "hour"], how="left")
#     ####
    
    
    
    return df
    
df_all = feature_engineering(df_all, 1)


In [8]:
df_all = feature_engineering(df_all, 7)

In [9]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35252 entries, 0 to 35251
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                35252 non-null  int64         
 1   date              35252 non-null  datetime64[ns]
 2   value             26517 non-null  float64       
 3   day               35252 non-null  int64         
 4   month             35252 non-null  int64         
 5   hour              35252 non-null  int64         
 6   year              35252 non-null  int64         
 7   week              35252 non-null  int64         
 8   dayofweek         35252 non-null  int64         
 9   dayofyear         35252 non-null  int64         
 10  quarter           35252 non-null  int64         
 11  weekofyear        35252 non-null  int64         
 12  value_1dayofyear  26418 non-null  float64       
 13  dayofyear_1d      26418 non-null  float64       
 14  value_7dayofyear  2591

In [10]:
business_hours = [9, 10, 11, 12, 13, 14, 15, 16, 17]
df_all["business_hour"]=df_all["hour"].map(lambda x: 1 if x in business_hours else 0)

In [11]:
df_all["hour_month"] = df_all.apply(lambda x: "{}-{}".format(x["hour"], x["month"]), axis=1)
df_all["hour_day"] = df_all.apply(lambda x: "{}-{}".format(x["hour"], x["day"]), axis=1)
df_all["hour_day_of_week"] = df_all.apply(lambda x: "{}-{}".format(x["hour"], x["dayofweek"]), axis=1)
df_all["hour_day_of_week_month"] = df_all.apply(lambda x: "{}-{}".format(x["hour_day_of_week"], x["month"]), axis=1)
# df_all["business_hour_day_of_week"] = df_all.apply(lambda x: "{}-{}".format(x["business_hour"], x["day_of_week"]), axis=1)

In [12]:
df_all["hour_month_cat"] = df_all["hour_month"].factorize()[0]
df_all["hour_day_cat"] = df_all["hour_day"].factorize()[0]
df_all["hour_day_of_week_cat"] = df_all["hour_day_of_week"].factorize()[0]
df_all["hour_day_of_week_month_cat"] = df_all["hour_day_of_week_month"].factorize()[0]
# df_all["business_hour_day_of_week_cat"] = df_all["business_hour_day_of_week"].factorize()[0]

In [13]:
def df_group_delay(df_train, groupby_feats):
    agg_params = {
        "mean_{}_value".format("_".join(groupby_feats)): ("value", "mean"),
        "median_{}_value".format("_".join(groupby_feats)): ("value", "median"),
        #"count_{}_value".format("_".join(groupby_feats)): ("value", "count"),
        #"std_{}_value".format("_".join(groupby_feats)): ("value", "std"),
        #"count_zeros_{}_value".format("_".join(groupby_feats)): ("value", lambda vals: len([x for x in vals if x == 0]) ),
        #"prob_zeros_{}_value".format("_".join(groupby_feats)): ("value", lambda vals: np.mean([x == 0 for x in vals]) ),
    }
    
    return df_train[groupby_feats + ["value"]].groupby(groupby_feats).agg(
        **agg_params
    ).reset_index()

# df_tmp = df_group_delay(df_train, ["hour"])
# if "mean_hour_value" not in df_all:
#     df_all = pd.merge(df_all, df_tmp, on="hour", how="left")

df_tmp = df_group_delay(df_all, ["hour_day_of_week_month"])
if "mean_hour_day_of_week_month_value" not in df_all:
    df_all = pd.merge(df_all, df_tmp, on="hour_day_of_week_month", how="left")
    
df_tmp = df_group_delay(df_all, ["value_1dayofyear"])
if "mean_value_1dayofyear_value" not in df_all:
    df_all = pd.merge(df_all, df_tmp, on="value_1dayofyear", how="left")

df_tmp = df_group_delay(df_all, ["value_7dayofyear"])
if "mean_value_7dayofyear_value" not in df_all:
    df_all = pd.merge(df_all, df_tmp, on="value_7dayofyear", how="left")

# df_tmp = df_group_delay(df_all, ["hour_month"])
# if "mean_hour_month_value" not in df_all:
#     df_all = pd.merge(df_all, df_tmp, on="hour_month", how="left" )
    
# df_tmp = df_group_delay(df_all, ["hour_day_of_week"])
# if "mean_hour_day_of_week_value" not in df_all:
#     df_all = pd.merge(df_all, df_tmp, on="hour_day_of_week", how="left" )

In [14]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35252 entries, 0 to 35251
Data columns (total 31 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   id                                   35252 non-null  int64         
 1   date                                 35252 non-null  datetime64[ns]
 2   value                                26517 non-null  float64       
 3   day                                  35252 non-null  int64         
 4   month                                35252 non-null  int64         
 5   hour                                 35252 non-null  int64         
 6   year                                 35252 non-null  int64         
 7   week                                 35252 non-null  int64         
 8   dayofweek                            35252 non-null  int64         
 9   dayofyear                            35252 non-null  int64         
 10  quarter   

In [ ]:
# group_features = {
#         "group_mean_hour_dayofweek": ["hour", "dayofweek"],
#         "group_mean_hour_month": ["hour", "month"],
#         "group_mean_hour_dayofweek_month": ["hour", "dayofweek", "month"],
#     }
    

#     for feat_name,group_keys in group_features.items(): 
#         dict_group = {k:v["value"] for k,v in df_train[["value"] + group_keys].groupby(group_keys).agg("mean").to_dict(orient="items").items() }
#         df[feat_name] = df[ group_keys ].apply(lambda keys: dict_group.get(tuple(keys)), axis=1)
    

In [15]:
qgrid.show_grid(df_all, show_toolbar=True)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [95]:
df_all.corr("pearson")

,id,value,day,month,hour,year,week,dayofweek,dayofyear,quarter,...,hour_month_cat,hour_day_cat,hour_day_of_week_cat,hour_day_of_week_month_cat,mean_hour_day_of_week_month_value,median_hour_day_of_week_month_value,mean_value_1dayofyear_value,median_value_1dayofyear_value,mean_value_7dayofyear_value,median_value_7dayofyear_value
id,1.000000,0.106168,0.004767,0.033852,0.000610,0.969923,0.042208,-0.001706,0.036130,0.039358,...,0.229327,0.004704,0.000066,0.233310,0.067020,0.064023,0.129553,0.129573,0.101775,0.101769
value,0.106168,1.000000,0.029687,-0.023231,0.500412,0.107406,-0.030478,-0.333319,-0.019724,-0.028640,...,0.284717,0.029391,0.266159,0.252619,0.921625,0.913786,0.997293,0.997279,0.998946,0.998929
day,0.004767,0.029687,1.000000,0.017658,0.000269,-0.019940,0.070373,0.004364,0.101029,0.021647,...,-0.002752,0.790468,-0.003576,-0.003050,-0.006670,-0.004671,0.056231,0.056270,0.048385,0.048364
month,0.033852,-0.023231,0.017658,1.000000,-0.000192,-0.209583,0.966367,-0.004219,0.996487,0.971325,...,0.180954,0.011481,0.003335,0.180822,-0.023547,-0.018232,-0.021931,-0.021901,-0.008146,-0.008109
hour,0.000610,0.500412,0.000269,-0.000192,1.000000,-0.000028,-0.000176,0.000283,-0.000170,-0.000253,...,0.082970,0.032343,0.142624,0.011584,0.546909,0.543118,0.495727,0.495785,0.497763,0.497756
year,0.969923,0.107406,-0.019940,-0.209583,-0.000028,1.000000,-0.194360,-0.000742,-0.208207,-0.197344,...,0.179325,-0.014276,-0.000765,0.183307,0.070745,0.066517,0.128047,0.128057,0.098824,0.098809
week,0.042208,-0.030478,0.070373,0.966367,-0.000176,-0.194360,1.000000,-0.000350,0.967591,0.943060,...,0.192288,0.064385,0.000729,0.192915,-0.021030,-0.015248,-0.032636,-0.032604,-0.008464,-0.008428
dayofweek,-0.001706,-0.333319,0.004364,-0.004219,0.000283,-0.000742,-0.000350,1.000000,-0.003812,-0.004484,...,0.003982,0.001322,-0.494708,-0.002950,-0.360800,-0.360373,-0.335118,-0.335082,-0.333967,-0.333936
dayofyear,0.036130,-0.019724,0.101029,0.996487,-0.000170,-0.208207,0.967591,-0.003812,1.000000,0.968377,...,0.185133,0.077432,0.003029,0.184983,-0.022631,-0.017307,-0.016190,-0.016156,-0.003115,-0.003079
quarter,0.039358,-0.028640,0.021647,0.971325,-0.000253,-0.197344,0.943060,-0.004484,0.968377,1.000000,...,0.217951,0.014290,0.004102,0.217831,-0.028227,-0.024918,-0.025375,-0.025338,-0.013554,-0.013518


# Preparing models

In [16]:
# feats = ["day", "month", "hour", "hour_month_cat", "hour_day_cat", "day_of_week",  

feats = df_all.select_dtypes("number").columns
black_list = ["id", "hour_day_of_week_cat" "hour_day_of_week_cat", "count_zeros_hour_day_of_week_month_cat_value",
             "prob_zeros_hour_day_of_week_month_cat_value", "count_hour_day_of_week_month_cat_value", 
             "count_hour_day_of_week_month_cat_value","quarter", "hour_day_cat", "value", "count_hour_day_of_week_month_value",
             "count_hour_month_value", "count_hour_day_of_week_value", "dayofyear_1d", "dayofweek_1d", "dayofyear_7d"]
             

feats = [x for x in feats if x not in black_list]
feats

['day',
 'month',
 'hour',
 'year',
 'week',
 'dayofweek',
 'dayofyear',
 'weekofyear',
 'value_1dayofyear',
 'value_7dayofyear',
 'business_hour',
 'hour_month_cat',
 'hour_day_of_week_cat',
 'hour_day_of_week_month_cat',
 'mean_hour_day_of_week_month_value',
 'median_hour_day_of_week_month_value',
 'mean_value_1dayofyear_value',
 'median_value_1dayofyear_value',
 'mean_value_7dayofyear_value',
 'median_value_7dayofyear_value']

In [17]:
df_train = df_all[ df_all["value"].notnull() ]
df_test = df_all[ df_all["value"].isnull() ]

X_train = df_train[feats].fillna(-1).values
y_train = df_train["value"].values
X_test = df_test[feats].fillna(-1).values

# Train and forecast

In [18]:
#model = DecisionTreeRegressor()
model = xgb.XGBRegressor(max_depth=8, n_estimators=50, random_state=0)
model.fit(X_train, y_train)
df_test["value"] = model.predict(X_test)

<ipython-input-18-adb76fd5a786>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["value"] = model.predict(X_test)


In [19]:
scores = cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_absolute_error")
print("local score: ", np.mean(scores), np.std(scores))

local score:  -207.30260958388718 15.943383583780172


# To Kaggle

In [100]:
df_test[ ["id", "value"] ].to_csv("../output/final_result.csv", index=False)